### The `requests` Library

https://requests.readthedocs.io/en/master/

In [1]:
import requests

Let's first look at how we can emulate the `GET` requests that browsers make when we request pages from web sites.

Here we're going to pull the home page of the site `https://www.stackoverflow.com` - since we don't specify anything in the URL beyond just the domain, we'll get whatever "default" page is defined by that site:

In [2]:
response = requests.get("https://www.stackoverflow.com")

Now we can look at the status code/message returned in the `response` object:

In [3]:
response.status_code, response.reason

(200, 'OK')

As you can see, the status was `200/OK` - which means our request was successful.

The response is going to be HTML, not JSON, so we can recover it using the `text` property:

In [4]:
response.text

'\n<!DOCTYPE html>\n\n\n    <html class="html__responsive " lang="en">\n\n    <head>\n\n        <title>Newest Questions - Stack Overflow</title>\n        <link rel="shortcut icon" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/favicon.ico?v=ec617d715196">\n        <link rel="apple-touch-icon" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a">\n        <link rel="image_src" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a"> \n        <link rel="search" type="application/opensearchdescription+xml" title="Stack Overflow" href="/opensearch.xml">\n    <meta name="viewport" content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0">\n        <meta property="og:type" content= "website" />\n        <meta property="og:url" content="https://stackoverflow.com/questions"/>\n        <meta property="og:site_name" content="Stack Overflow" />\n        <meta property="og:image" itempro

This is the HTML your browser would display after it does it's `GET` request.

Responses usually also contain **headers** - we don't see those in our browsers, but it may affect how the browser behaves. Using `requests`, the headers are returned as a `dict`:

In [5]:
for key, value in response.headers.items():
    print(f'{key}: \t{value}')

Date: 	Wed, 18 Jun 2025 05:12:37 GMT
Content-Type: 	text/html; charset=utf-8
Transfer-Encoding: 	chunked
Connection: 	keep-alive
CF-Ray: 	95184af02fa2ff6a-BOM
CF-Cache-Status: 	DYNAMIC
Cache-Control: 	private
Set-Cookie: 	prov=d302b3e9-d35d-4cb8-aa03-22f951ac593e; expires=Thu, 18 Jun 2026 05:12:37 GMT; domain=.stackoverflow.com; path=/; secure; samesite=none; httponly
Strict-Transport-Security: 	max-age=31536000; includeSubDomains
Vary: 	Accept-Encoding
content-security-policy: 	upgrade-insecure-requests; frame-ancestors 'self' https://stackexchange.com
feature-policy: 	microphone 'none'; speaker 'none'
x-frame-options: 	SAMEORIGIN
x-request-guid: 	c98f83e7-88b6-4009-8a11-da7af8bdf783
x-worker-origin-response-time: 	283000000
X-DNS-Prefetch-Control: 	off
Server: 	cloudflare
Content-Encoding: 	gzip


Notice one of the keys in this dictionary: `content-type` - it tells the browser that the content is **html** and the unicode character encoding is `utf-8`. Notice also that `content-encoding` has a value of `gzip` - this is because the content was actually compressed by the server befopre sending out the response - and in this case both `requests` and our browser would decompress the response - this helps keep the amount of data transmitted over the web as compact as possible.

We usually don't do much with response headers, although some APIs may return extra information in the headers (maybe how long the request took to compute on the server side) - these are going to be API specific, so you'll need to read your API docs.

Next, let's try using a `GET` request with Google's search - although Google provides an API for search (you'll have to sign up to get a security token), we're just going to simulate using a GET request the way your browser does, for example:

- `https://www.google.com` --> google search home page
- `https://www.google.com?q=text to search` --> searches "text to search"

In [6]:
query_params = {
    'q': 'python http requests',
    'num': 5
}
response = requests.get(
    "https://www.google.com/search", 
    params=query_params
)

In [7]:
response.status_code, response.reason

(200, 'OK')

Once more our request was succesful, and the response returned is HTML:

In [8]:
response.headers['content-type']

'text/html; charset=ISO-8859-1'

In [9]:
response.headers['content-encoding']

'gzip'

In [10]:
response.text

'<!DOCTYPE html><html lang="en-IN"><head><title>Google Search</title><style>body{background-color:#fff}</style></head><body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=yUpSaLHaKpmO4-EP1vL_sA8" http-equiv="refresh"><div style="display:block">Please click <a href="/httpservice/retry/enablejs?sei=yUpSaLHaKpmO4-EP1vL_sA8">here</a> if you are not redirected within a few seconds.</div></noscript><script nonce="tc-r2odwthoTrGX1Rcy43w">//# sourceMappingURL=data:application/json;charset=utf-8;base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOlsiICJdLCJuYW1lcyI6WyJjbG9zdXJlRHluYW1pY0J1dHRvbiJdLCJtYXBwaW5ncyI6IkFBQUE7QUFBQTtBQUFBO0FBQUE7QUFBQTtBQUFBO0FBQUEifQ==\n(function(){function S(B){return B}var P=function(B,z,v,A,R,w,g,F,r,a,M,d){for(M=(d=90,98);;)try{if(d==66)break;else if(d==90)r=g,F=h.trustedTypes,d=z;else if(d==A)M=98,d=B;else{if(d==54)return r;if(d==z)d=F&&F.createPolicy?v:54;else if(d==v)M=14,r=F.createP

Note that if we try to read the `json` property we would get an exception since this response does not contain JSON data:

In [11]:
try:
    response.json()
except ValueError as ex:
    print(ex)

Expecting value: line 1 column 1 (char 0)


Also, if we make a bad request, such as an unkown resource path, we'll get a different response `status_code`, not a Python exception:

In [12]:
response = requests.get('https://www.google.com/some_dummy_page')

In [13]:
response.status_code, response.reason

(404, 'Not Found')

We can force a Python exception, by using the `raise_for_status` method on the response object:

In [14]:
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://www.google.com/some_dummy_page

So if we want a Python exception to occur when a status code is not in the `2xx` range, we can do this:

In [15]:
try:
    response = requests.get('https://www.google.com/bogus')
    response.raise_for_status()
except requests.HTTPError as ex:
    print(ex)

404 Client Error: Not Found for url: https://www.google.com/bogus


But if the request is succesful, no exception will be raised:

In [16]:
try:
    response = requests.get('https://www.google.com')
    response.raise_for_status()
except requests.HTTPError as ex:
    print(ex)

One other piece of information that may be interesting in the response, is the `cookies` attribute where we can see what cookies are being saved to our machine. Again, it is a dictionary:

In [17]:
response = requests.get('http://www.nyt.com')
for key, value in response.cookies.items():
    print(f'{key}: {value}')

nyt-a: c5H_CDk53ILzf0zWvCRqOL
nyt-gdpr: 0
nyt-purr: cfhhcfhhhckfhdfshgas2fdnd
nyt-geo: IN
nyt-traceid: 00000000000000002d2bfba080811432


Now, let's look at using a REST API with JSON data. As we covered in the previous set of lectures, you'll need access to some REST API and most likely will need an authentication token.

For our examples, we are going to use the REST API provided by:

https://www.finnhub.io/

You'll need to retrieve your API key from that site after you log in (discussed in previous lecture on REST APIs).

In [33]:
API_KEY = 'd194m61r01qkcat6208gd194m61r01qkcat62090'

### CAUTION!!

This API key is a **secret** - do not share it with anyone, and most definitely do not post it on the web anywhere! Be careful to remove it if you share your Jupyter notebook/Python code with someone,s or use `githib` for source control.

A simple approach (not perfect, but better than nothing), would be to save your token in a file, and read the token back into your notebook/Python code from that file. Then, when you share the notebook, you simply don't share the "secret" file.

Here, I have a file called "secrets.txt" that contains a single line, my API token, and I can read it back this way:

In [34]:
with open('secrets.txt') as f:
    API_KEY = next(f).strip()

According to `finnhub.io`'s documentation, we can pass that token either as a query argument (`http://finnhub.io/api/v1/...?token=apiKey`), or we can pass it in the headers using the key `X-Finnhub-Token`.

Let's try it both ways:

In [35]:
base_url = 'https://finnhub.io/api/v1/'

We're going to retrieve stock quotes - see docs here: https://finnhub.io/docs/api#quote

In [36]:
url = f'{base_url}/quote'
params = {'symbol': 'AAPL', 'token': API_KEY}
response = requests.get(url, params)

In [37]:
response.status_code

401

In [38]:
response.headers['content-type']

'application/json'

As we can see, the response content type is JSON, and we can use the `json()` method from requests:

In [39]:
response.json()

{'error': 'Invalid API key.'}

`requests` automatically deserialized the JSON response into a Python `dict`:

In [40]:
data = response.json()
data['o']

KeyError: 'o'

The string that `requests` deserializes is actually the one contained in the `text` attribute of the response:

In [26]:
response.text

'{"error":"Invalid API key."}'

It's just handy that `requests` will do the `loads` for us, essentially. But, if we prefer we can do it manually as well, deserializing the `text` attribute's value.

You'll notice that there is a `t` field returned in the JSON:

In [27]:
time = data['t']

KeyError: 't'

In [28]:
time

CPU times: total: 0 ns
Wall time: 0 ns


This is a POSIX epoch timestamp, so we can use Python to convert this epoch time into a more useful `datetime`:

In [41]:
from datetime import datetime

In [42]:
d = datetime.fromtimestamp(time)

NameError: name 'time' is not defined

In [43]:
d

NameError: name 'd' is not defined

Now if we want to query another symbol, we just have to change our query parameter - however, since we are passing our token in the query params, we need to provide it as well:

In [44]:
params = {
    'symbol': 'MSFT',
    'token': API_KEY
}
response = requests.get(url=url, params=params)
response.raise_for_status()

HTTPError: 401 Client Error: Unauthorized for url: https://finnhub.io/api/v1//quote?symbol=MSFT&token=put+your+token+here

In [45]:
response.json()

{'error': 'Invalid API key.'}

This means evey time we change our params, we have to remember to add our API token to it. Instead, we can use the headers, and not have to remember to pass it in the params every time:

In [46]:
headers = {
    'X-Finnhub-Token': API_KEY
}
for symbol in ['AAPL', 'MSFT', 'GOOG']:
    try:
        response = requests.get(
            url, 
            params={'symbol': symbol}, 
            headers=headers
        )
        response.raise_for_status()
        print(f'***** {symbol} *****')
        print(response.json())
    except requests.HTTPError as ex:
        print('Unable to retrieve data: {ex}')

Unable to retrieve data: {ex}
Unable to retrieve data: {ex}
Unable to retrieve data: {ex}


Let's look at one more, the `Company Profile 2` resource:

In [47]:
url = f'{base_url}/stock/profile2'
response = requests.get(
    url, 
    params={'symbol': 'AAPL'},
    headers=headers
)
response.json()

{'error': 'Invalid API key'}

Finally, let's see how we can use `POST`. The choices for `POST` calls to this API are quite limited (it is mainly a service for retrieving data, not creating resources).

However, they implement something called a **webhook**. Basically, you can define (in your Finnhub dashboard) a URL (on your server) that will be called when certain events happen on their end - in this case they have support for real-time earnings updates.

We don't have a web server available here for the callback, but we can still manage those webhooks, creating, listing and deleting them, so let's try that, even though they won't be fully "functional".

To create a webhook, we need to use the API defined here: https://finnhub.io/docs/api#webhook

In [48]:
base_webhook_url = f'{base_url}/webhook'

Let's create a couple of webhooks for receiving earnings updates for `AAPL` and `MSFT`:

In [49]:
post_data = {
    'event': 'earnings',
    'symbol': 'AAPL'
}

In [50]:
response = requests.post(
    url=f'{base_webhook_url}/add',
    headers=headers,
    json=post_data
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


And let's add a second one for `MSFT`:

In [51]:
response = requests.post(
    url=f'{base_webhook_url}/add',
    headers=headers,
    json={
        'event': 'earnings',
        'symbol': 'MSFT'
    }
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


Now let's read those webhooks back. According to the API docs, we have to issue a `GET` request to `.../webhook/list`, so let's do that:

In [52]:
response = requests.get(
    url=f'{base_webhook_url}/list',
    headers=headers
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


Finally, let's delete those webhooks. You'll notice that we have an `id` field associated with each webhook - this is the unique identifier for our webhooks, and we'll need to use that `id` value to specify which webhook we want to delete.

The API docs specify that to delete a webhook we need to use a `POST` to the URL `.../webhook/delete` and pass the `id` in the JSON data we send.

This is a bit unorthodox, since `DELETE` is an actual HTTP method and is more traditional for deleting resources, but as we discussed before, all this is entirely up to the API developers.

**Note**: I am just typing the id value directly in this request, you will therefore need to modify that id to match the webhooks you just created.

In [53]:
response = requests.post(
    url=f'{base_webhook_url}/delete',
    headers=headers,
    json={
        'id': 3190
    }
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


So this webhook was deleted succesfully - let's check by getting a list of all webhooks:

In [54]:
response = requests.get(
    url=f'{base_webhook_url}/list',
    headers=headers
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


As you can see, only one left. Let's delete that last one:

In [55]:
response = requests.post(
    url=f'{base_webhook_url}/delete',
    headers=headers,
    json={
        'id': 3191
    }
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}


In [56]:
response = requests.get(
    url=f'{base_webhook_url}/list',
    headers=headers
)
print(response.status_code, response.reason)
print(response.json())

401 Unauthorized
{'error': 'Invalid API key'}
